In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import matplotlib
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.stattools as st
from statsmodels.tsa.arima_model import ARIMA
import warnings
import time
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm

D:\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
warnings.filterwarnings('ignore')
matplotlib.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['backend'] = 'SVG'

In [3]:
df = pd.read_excel('E:/论文/论文2/数据/表.xlsx',index_col=0)
df.reset_index(inplace=True)
df.head()

,Date,USD (PM),Value,DealDay,黄金涨幅(%),比特币涨幅(%),黄金5日均价,黄金5日乖离率,比特币5日均价,比特币5日乖离率,...,黄金涨幅的前90日均值,黄金5日乖离率的前90日均值,黄金牛市,gold_cow,比特币涨幅的前30日均值,比特币5日乖离率的前30日均值,比特币牛市,bt_cow,黄金购买风险,比特币购买风险
0,2016-09-11,0.00,621.65,0,0.493311,0.641569,0.0,0.460623,0.000000,0.550392,...,NaN,NaN,0.494444,-1,NaN,NaN,0.483333,-1,0.492608,0.485020
1,2016-09-12,1324.60,609.67,1,0.493311,0.609981,0.0,0.460623,0.000000,0.550392,...,NaN,NaN,0.488889,-2,NaN,NaN,0.466667,-2,0.487354,0.468772
2,2016-09-13,1323.65,610.92,1,0.486412,0.644930,0.0,0.460623,0.000000,0.550392,...,NaN,NaN,0.483333,-3,NaN,NaN,0.450000,-3,0.482100,0.452525
3,2016-09-14,1321.75,608.82,1,0.479503,0.635935,0.0,0.460623,0.000000,0.550392,...,NaN,NaN,0.477778,-4,NaN,NaN,0.433333,-4,0.476846,0.436278
4,2016-09-15,1310.80,610.38,1,0.413621,0.645769,0.0,0.460623,0.009834,0.545356,...,NaN,NaN,0.472222,-5,NaN,NaN,0.416667,-5,0.471592,0.419903


In [4]:
df.shape

(1826, 29)

使用SARIMAX模型进行预测，参数选择来自于02中的代码

使用SARIMAX(p,d,q)$\times$(P,D,Q,s)模型，数据量不能少于$d + D*s + max(3*q + 1, 3*Q*s + 1, p, P*s) + 1$

In [5]:
# 计算SARIMAX最少需要的样本数
def num_of_obj(p,d,q,P,D,Q,s):
    nobs = d + D*s + max([3*q + 1, 3*Q*s + 1, p, P*s]) + 1
    return nobs

In [6]:
def gold_predict(data,date):
    #model = ARIMA(data, order=(6,1,6))  # order(p,d阶差分，q)
    model = SARIMAX(data, order=(6,1,6))
    result = model.fit()
    try:
        pred = result.predict(date.strftime('%m-%d-20%y'),(date+datetime.timedelta(5)).strftime('20%y-%m-%d'))
    except:
        # 5天后可能不是交易日会报错，加两天
        pred = result.predict(date.strftime('%m-%d-20%y'),(date+datetime.timedelta(7)).strftime('20%y-%m-%d'))
    return pred, abs(sum(result.resid))

In [7]:
def bt_predict(data,date):
    #nobs = num_of_obj(12,1,12,0,0,0,0)
    if len(data)>=62:
        #model = ARIMA(data, order=(12,1,12))
        model = SARIMAX(data[-62:], order=(12,1,12))
        result = model.fit()
        pred = result.predict(date.strftime('%m-%d-20%y'),(date+datetime.timedelta(5)).strftime('20%y-%m-%d'))
    return pred, abs(sum(result.resid))

In [8]:
df['黄金价格残差']=0
df['比特币价格残差']=0
df['预计黄金价格']=0
df['预计比特币价格']=0
df['预计黄金涨幅']=0
df['预计比特币涨幅']=0
bt_df = df[['Date','Value']]
gold_df = df[['Date','USD (PM)']][df['DealDay']==1]
gold_df.index=gold_df['Date']
bt_df.index=bt_df['Date']
del gold_df['Date']
del bt_df['Date']

In [9]:
print('黄金最少样本数:',num_of_obj(6,1,6,0,0,0,0))
print('比特币最少样本数:',num_of_obj(12,1,12,0,0,0,0))

黄金最少样本数: 21
比特币最少样本数: 39


## 这里会运行3949.6963279247284秒

In [10]:
t=time.time()
for i in range(df.shape[0]):
    print('i:',i)
    if df['DealDay'][i]:
        try:
            p,r=gold_predict(gold_df[:i],df['Date'][i])
            df['黄金价格残差'][i] = r
            #df['预计黄金涨幅'][i] = p[0]
            df['预计黄金价格'][i] = p[0]
            print('\t预计黄金价格:',p[0],'\t黄金价格残差:',r)
        except:
            pass
    try:
        p,r=bt_predict(bt_df[:i],df['Date'][i])
        df['比特币价格残差'][i] = r
        #df['预计比特币涨幅'][i] = p[0]
        df['预计比特币价格'][i] = p[0]
        print('\t预计比特币价格:',p[0],'\t比特币价格残差:',r,'\n')
    except:
        pass
time.time()-t

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
	预计黄金价格: 1251.758948606603 	黄金价格残差: 1281.4094169312457
i: 27
i: 28
i: 29
	预计黄金价格: 1261.3364474360592 	黄金价格残差: 1299.7274833844267
i: 30
	预计黄金价格: 1249.9793724751978 	黄金价格残差: 1296.1575317487034
i: 31
i: 32
	预计黄金价格: 1257.6098452278422 	黄金价格残差: 1304.04358892154
i: 33
	预计黄金价格: 1262.6033136064075 	黄金价格残差: 1298.5785652506565
i: 34
i: 35
i: 36
	预计黄金价格: 1249.1538530318041 	黄金价格残差: 1283.5911379147658
i: 37
	预计黄金价格: 1258.1489747412425 	黄金价格残差: 1311.678821311684
i: 38
	预计黄金价格: 1262.5986582391997 	黄金价格残差: 1322.9551493843874
i: 39
	预计黄金价格: 1276.9895933944842 	黄金价格残差: 1317.6481728352721
i: 40
	预计黄金价格: 1265.170447964634 	黄金价格残差: 1317.225954286662
i: 41
i: 42
i: 43
	预计黄金价格: 1267.4777475484343 	黄金价格残差: 1268.8102841304708
i: 44
	预计黄金价格: 1266.7089926992428 	黄金价格残差: 1262.9283378271205
i: 45
	预计黄金价格: 1271.7783182148785 	黄金价格残差: 1262.292997134003
i: 46
	预计黄金价格

	预计比特币价格: 722.8574480282675 	比特币价格残差: 774.7732517756851 

i: 129
	预计黄金价格: 1215.8025720086528 	黄金价格残差: 1258.2750863986091
i: 130
	预计黄金价格: 1218.8672664617227 	黄金价格残差: 1261.2131849896552
i: 131
	预计黄金价格: 1195.6239724695802 	黄金价格残差: 1259.9808263661823
	预计比特币价格: 864.70217498518 	比特币价格残差: 872.8336189544226 

i: 132
	预计比特币价格: 980.8832546305212 	比特币价格残差: 969.1664152257927 

i: 133
	预计比特币价格: 949.8582052789168 	比特币价格残差: 966.3296508729053 

i: 134
	预计黄金价格: 1197.3323563243084 	黄金价格残差: 1262.9451333514173
i: 135
	预计黄金价格: 1217.5313841917266 	黄金价格残差: 1261.6315895528592
i: 136
	预计黄金价格: 1214.158587694007 	黄金价格残差: 1271.7895738142283
	预计比特币价格: 931.8666241328274 	比特币价格残差: 969.5170087453096 

i: 137
	预计黄金价格: 1187.3695083105918 	黄金价格残差: 1263.3967894194463
	预计比特币价格: 902.9750668201225 	比特币价格残差: 890.2356504304274 

i: 138
	预计黄金价格: 1192.1142211635004 	黄金价格残差: 1259.1515746008654
i: 139
	预计比特币价格: 930.7868004339886 	比特币价格残差: 940.2834421943747 

i: 140
	预计比特币价格: 917.0234829466465 	比特币价格残差: 944.9788166434271 

i: 141


	预计比特币价格: 1230.1034145111423 	比特币价格残差: 1955.4939944491316 

i: 214
	预计黄金价格: 1278.1233607980364 	黄金价格残差: 1257.5661421941425
	预计比特币价格: 1259.140461543271 	比特币价格残差: 2864.590735054973 

i: 215
	预计比特币价格: 1211.389003756579 	比特币价格残差: 2773.851866516845 

i: 216
	预计比特币价格: 1210.4322974099034 	比特币价格残差: 3328.837373694876 

i: 217
	预计比特币价格: 1182.5219578348826 	比特币价格残差: 2785.437438188519 

i: 218
	预计比特币价格: 1300.037481560268 	比特币价格残差: 3071.1744107774352 

i: 219
	预计黄金价格: 1282.8031904783525 	黄金价格残差: 1263.5769881210608
	预计比特币价格: 1213.209419866689 	比特币价格残差: 1202.5374370208388 

i: 220
	预计黄金价格: 1276.2075481909337 	黄金价格残差: 1274.4730119495293
	预计比特币价格: 1135.4841468405546 	比特币价格残差: 870.6436190936981 

i: 221
	预计黄金价格: 1281.7826461823415 	黄金价格残差: 1276.8335942262709
	预计比特币价格: 1203.9277788658455 	比特币价格残差: 1129.4763347614212 

i: 222
	预计黄金价格: 1287.1092089126143 	黄金价格残差: 1278.65221087441
	预计比特币价格: 1197.6721589154497 	比特币价格残差: 1002.4706161013503 

i: 223
	预计比特币价格: 1174.8197688715973 	比特币价格残差: 1028.0645178309583 

i

i: 300
i: 301
	预计比特币价格: 2504.1832227148775 	比特币价格残差: 1389.0309702616137 

i: 302
	预计黄金价格: 1211.3179420958388 	黄金价格残差: 1284.341800707498
	预计比特币价格: 2422.637194496076 	比特币价格残差: 2271.740796797002 

i: 303
	预计黄金价格: 1212.6431318632585 	黄金价格残差: 1291.1342806655337
	预计比特币价格: 2491.409527959547 	比特币价格残差: 1062.0212472960839 

i: 304
	预计黄金价格: 1212.447682877008 	黄金价格残差: 1297.4433576264892
	预计比特币价格: 2209.649087485308 	比特币价格残差: 987.9865538434506 

i: 305
	预计黄金价格: 1218.1720614242572 	黄金价格残差: 1294.9359290536322
	预计比特币价格: 2362.5964961615214 	比特币价格残差: 430.8656447619246 

i: 306
	预计黄金价格: 1220.7766515256033 	黄金价格残差: 1298.3979408096936
	预计比特币价格: 2397.400605595232 	比特币价格残差: 134.69771280164719 

i: 307
	预计比特币价格: 2346.7391390021116 	比特币价格残差: 2870.8481582989707 

i: 308
i: 309
	预计黄金价格: 1232.0027101203984 	黄金价格残差: 1282.1489615176245
i: 310
	预计黄金价格: 1233.3172116430073 	黄金价格残差: 1280.2818093221013
i: 311
	预计黄金价格: 1242.9722759599629 	黄金价格残差: 1278.9668614353332
	预计比特币价格: 2406.750010619429 	比特币价格残差: 4516.683868798621 


i: 392
i: 393
	预计黄金价格: 1261.0603656205967 	黄金价格残差: 1335.690954212928
	预计比特币价格: 4592.248308907646 	比特币价格残差: 5304.438269373253 

i: 394
	预计黄金价格: 1282.2644432851819 	黄金价格残差: 1338.0319730912443
i: 395
	预计黄金价格: 1291.8860499961736 	黄金价格残差: 1342.4113442159628
i: 396
	预计黄金价格: 1288.193093457797 	黄金价格残差: 1352.8215524800225
i: 397
	预计黄金价格: 1291.2900186848985 	黄金价格残差: 1338.3100047360538
	预计比特币价格: 5049.769388812801 	比特币价格残差: 4690.665829996814 

i: 398
	预计比特币价格: 5313.339878250492 	比特币价格残差: 6532.226293890184 

i: 399
	预计比特币价格: 5673.661786553586 	比特币价格残差: 5732.160795079584 

i: 400
	预计黄金价格: 1300.6852748905083 	黄金价格残差: 1329.4761178396936
	预计比特币价格: 5743.102799348071 	比特币价格残差: 5269.027877707408 

i: 401
	预计黄金价格: 1302.62520827776 	黄金价格残差: 1339.5489070470135
	预计比特币价格: 5799.860552393752 	比特币价格残差: 5304.181323424118 

i: 402
	预计黄金价格: 1285.6427347799627 	黄金价格残差: 1342.1322753568481
i: 403
	预计黄金价格: 1281.1076042736777 	黄金价格残差: 1333.299263444414
i: 404
	预计黄金价格: 1285.6625589465239 	黄金价格残差: 1325.4556254748434
i: 405

...

	预计比特币价格: 17618.900684786582 	比特币价格残差: 16107.334440009668 

i: 1541
	预计黄金价格: 1783.9063562919519 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 18043.999136025235 	比特币价格残差: 12475.907266662101 

i: 1542
	预计黄金价格: 1766.7005747059254 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 18205.278265158642 	比特币价格残差: 13029.254234783384 

i: 1543
	预计黄金价格: 1812.3436871573476 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 20530.58702324159 	比特币价格残差: 13399.343709824907 

i: 1544
	预计黄金价格: 1823.380633245532 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 19138.944364284733 	比特币价格残差: 15063.83399125589 

i: 1545
	预计黄金价格: 1834.7312217827243 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 19593.88472781193 	比特币价格残差: 13003.677368069115 

i: 1546
	预计比特币价格: 19458.53160849076 	比特币价格残差: 13136.647099163112 

i: 1547
	预计比特币价格: 19384.416010602083 	比特币价格残差: 13822.694563447025 

i: 1548
	预计黄金价格: 1840.0053808292118 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 19520.101790572975 	比特币价格残差: 12858.837207435503 

i: 1549
	预计黄金价格: 1859.24291780269 	黄金价格残差: 1834.8201839650437
	预计

	预计黄金价格: 1787.5236916200151 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 54367.22359574237 	比特币价格残差: 37800.02144302086 

i: 1626
	预计黄金价格: 1810.8851687058113 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 58568.05086221807 	比特币价格残差: 36254.942961447705 

i: 1627
	预计黄金价格: 1803.1284266605192 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 54263.84971290119 	比特币价格残差: 36100.31735354307 

i: 1628
	预计黄金价格: 1787.9685283481713 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 46754.520408717566 	比特币价格残差: 34290.33173853087 

i: 1629
	预计黄金价格: 1774.5647480199295 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 52021.60498851681 	比特币价格残差: 33375.52827626255 

i: 1630
	预计比特币价格: 46555.87260193151 	比特币价格残差: 31864.27366209864 

i: 1631
	预计比特币价格: 48560.98377455967 	比特币价格残差: 21254.353760446134 

i: 1632
	预计黄金价格: 1738.5596262309025 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 45128.59047419959 	比特币价格残差: 23602.20820415415 

i: 1633
	预计黄金价格: 1732.445270711511 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 47074.30261210392 	比特币价格残差: 23053.4799417184 

i: 1634
	预计黄金价格: 1727

	预计黄金价格: 1828.1443858767034 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 57446.58464362161 	比特币价格残差: 61044.4923750556 

i: 1706
	预计黄金价格: 1818.6954764914117 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 50707.31425507893 	比特币价格残差: 55828.89801280151 

i: 1707
	预计比特币价格: 50484.81434576613 	比特币价格残差: 61922.650057362334 

i: 1708
	预计比特币价格: 48902.88093611638 	比特币价格残差: 52598.349040616275 

i: 1709
	预计黄金价格: 1840.6732560888997 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 47823.39457467074 	比特币价格残差: 66104.99868943047 

i: 1710
	预计黄金价格: 1855.90097398406 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 42506.74490309077 	比特币价格残差: 54403.3118273575 

i: 1711
	预计黄金价格: 1869.955730591512 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 45879.51127456608 	比特币价格残差: 50822.48655174515 

i: 1712
	预计黄金价格: 1887.4806264078072 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 42947.05166456559 	比特币价格残差: 42166.57424545385 

i: 1713
	预计黄金价格: 1876.207475224683 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 37298.86820755525 	比特币价格残差: 42783.33911444726 

i: 1714
	预计比特币价格: 41131.24

	预计比特币价格: 39199.0998558673 	比特币价格残差: 120671.41592733831 

i: 1785
	预计比特币价格: 42845.47472556493 	比特币价格残差: 43944.078812337895 

i: 1786
	预计黄金价格: 1828.26784107877 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 41939.59889981902 	比特币价格残差: 36715.16238618115 

i: 1787
	预计黄金价格: 1810.2160928040269 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 39098.34116203816 	比特币价格残差: 44318.43943460541 

i: 1788
	预计黄金价格: 1807.9707159314294 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 40006.63147087192 	比特币价格残差: 40678.14802587849 

i: 1789
	预计黄金价格: 1827.6955537302147 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 39091.173523742844 	比特币价格残差: 28278.931156010272 

i: 1790
	预计黄金价格: 1800.299868875615 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 37881.79529310983 	比特币价格残差: 32705.34734054392 

i: 1791
	预计比特币价格: 39127.01966593165 	比特币价格残差: 43910.1654931723 

i: 1792
i: 1793
	预计黄金价格: 1765.095706017463 	黄金价格残差: 1834.8201839650437
	预计比特币价格: 44556.41956370175 	比特币价格残差: 117316.34457492901 

i: 1794
	预计黄金价格: 1735.2703798589932 	黄金价格残差: 1834.8201839650437
	预计比特币价

3949.6963279247284

In [11]:
df

,Date,USD (PM),Value,DealDay,黄金涨幅(%),比特币涨幅(%),黄金5日均价,黄金5日乖离率,比特币5日均价,比特币5日乖离率,...,比特币牛市,bt_cow,黄金购买风险,比特币购买风险,黄金价格残差,比特币价格残差,预计黄金价格,预计比特币价格,预计黄金涨幅,预计比特币涨幅
0,2016-09-11,0.00,621.65,0,0.493311,0.641569,0.000000,0.460623,0.000000,0.550392,...,0.483333,-1,0.492608,0.485020,0,0,0,0,0,0
1,2016-09-12,1324.60,609.67,1,0.493311,0.609981,0.000000,0.460623,0.000000,0.550392,...,0.466667,-2,0.487354,0.468772,0,0,0,0,0,0
2,2016-09-13,1323.65,610.92,1,0.486412,0.644930,0.000000,0.460623,0.000000,0.550392,...,0.450000,-3,0.482100,0.452525,0,0,0,0,0,0
3,2016-09-14,1321.75,608.82,1,0.479503,0.635935,0.000000,0.460623,0.000000,0.550392,...,0.433333,-4,0.476846,0.436278,0,0,0,0,0,0
4,2016-09-15,1310.80,610.38,1,0.413621,0.645769,0.000000,0.460623,0.009834,0.545356,...,0.416667,-5,0.471592,0.419903,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2021-09-06,1821.60,51769.06,1,0.482234,0.701352,0.891906,0.463616,0.802853,0.607951,...,0.533333,2,0.477009,0.535210,0,34564,0,49894,0,0
1822,2021-09-07,1802.15,52677.40,1,0.390603,0.670330,0.890888,0.368978,0.815106,0.611716,...,0.516667,1,0.477124,0.519057,0,43332,0,51919,0,0
1823,2021-09-08,1786.00,46809.17,1,0.407108,0.458969,0.887195,0.321490,0.807012,0.439799,...,0.500000,0,0.479799,0.498486,0,39670,0,52113,0,0
1824,2021-09-09,1788.25,46078.38,1,0.505429,0.615979,0.883722,0.372245,0.794302,0.440009,...,0.483333,-1,0.487809,0.482244,0,32670,0,46629,0,0


In [14]:
# 黄金在非交易日的价格以上一个交易日的价格进行补充
df['预计黄金价格'].replace(0, np.nan,inplace=True)
df['预计黄金价格'].fillna(method='ffill',inplace=True)
df['预计比特币价格'].replace(0, np.nan,inplace=True)
df['预计比特币价格'].fillna(method='ffill',inplace=True)

tmp1_gold = df['USD (PM)'].copy() # 前一天的价格
tmp2_gold = df['预计黄金价格'].copy() # 当天预计的价格
tmp1_gold.index = tmp1_gold.index + 1
df['预计黄金涨幅'] = (tmp2_gold-tmp1_gold)/tmp1_gold

tmp1_bitcoin = df['Value'].copy() # 前一天的价格
tmp2_bitcoin = df['预计比特币价格'].copy() # 当天的价格
tmp1_bitcoin.index = tmp1_bitcoin.index + 1
df['预计比特币涨幅'] = (tmp2_bitcoin-tmp1_bitcoin)/tmp1_bitcoin

In [15]:
df.to_excel('E:/论文/论文2/数据/预计涨幅_改3.xlsx',index=False)